In [1]:
from dotenv import load_dotenv
import os
from langchain.llms import OpenAI
load_dotenv()
import warnings
warnings.filterwarnings("ignore")

In [2]:
## temperature value --> how creative we want our model to be
## 0 --> temperature it means model is very safe it is not taking any bets.
## 1 --> temperature  it will take risk it might generate wrong output but it is very creative.

In [3]:
llm = OpenAI(openai_api_key=os.getenv("openai_key"),temperature=0.7)

In [4]:
text = '''Which are the five best places in India and i want point wise answer i do not want
detail description i just want names'''
print(llm.predict(text))



1. Goa
2. Kerala
3. Rajasthan
4. Himachal Pradesh
5. Uttarakhand


## Prompt Templates and LLMChain

In [5]:
from langchain.prompts import PromptTemplate

input_prompt = PromptTemplate(
    input_variables=['country'],
    template = '''Give me the names of 5 best movies of {country} with highest box office collection 
    and also give the total worldwide collection in INR'''
)

print(input_prompt.format(country='India'))

Give me the names of 5 best movies of India with highest box office collection 
    and also give the total worldwide collection in INR


In [6]:
# In order to execute a prompt template you need to use LLMChain
from langchain.chains import LLMChain
chain = LLMChain(llm=llm,prompt=input_prompt)
print(chain.run("Korea"))



1. "Parasite" - Worldwide collection: INR 3,864 Crores
2. "Train to Busan" - Worldwide collection: INR 267 Crores
3. "Along with the Gods: The Two Worlds" - Worldwide collection: INR 1,080 Crores
4. "Extreme Job" - Worldwide collection: INR 578 Crores
5. "Veteran" - Worldwide collection: INR 348 Crores


## Combining Multiple Chains using Sequential Chain

In [7]:
# Simple Sequential Chain
# from langchain.chains import SimpleSequentialChain

# country_prompt = PromptTemplate(input_variables=['country'],
#                                 template='''Give me the name of richest man of {country} ''')
# country_chain = LLMChain(llm=llm,prompt=country_prompt,output_key='Person_Name')

# networth_prompt = PromptTemplate(input_variables=['networth'],
#                                  template='''And also tell me the networth of that person is {networth} ''')
# networth_chain = LLMChain(llm=llm,prompt=networth_prompt,output_key='Net_Worth')

In [8]:
# main_chain = SimpleSequentialChain(chains=[country_chain,networth_chain])
# main_chain.run("US")

In [9]:
# Sequential Chain
from langchain.chains import SequentialChain
country_prompt = PromptTemplate(input_variables=['country'],
                                template='''Tell me the richest man of {country} and i just want one name only ''')
country_chain = LLMChain(llm=llm,prompt=country_prompt,output_key='name')

networth_prompt = PromptTemplate(input_variables=['name'],
                                 template='''net worth of the {name}''')
networth_chain = LLMChain(llm=llm,prompt=networth_prompt,output_key='Net_Worth')


my_chain = SequentialChain(chains=[country_chain,networth_chain],
                        input_variables=['country'],
                        output_variables=['name','Net_Worth'])


In [10]:
my_chain({'country':'USA'})

{'country': 'USA',
 'name': '\n\nAs of 2021, the richest man in the USA is Elon Musk, with a net worth of approximately $209 billion.',
 'Net_Worth': ''}

## Chatmodels with ChatOpenAI

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [12]:
chat_llm = ChatOpenAI(openai_api_key=os.getenv("openai_key"),temperature=0.7,model='gpt-3.5-turbo')

In [16]:
chat_llm([
    SystemMessage(content="You are a professional singer AI assistant"),
    HumanMessage(content="Please write a song on Lord Hanuman")
])

AIMessage(content="(Verse 1)\nIn the realm of devotion, a hero was born,\nWith strength and wisdom, his spirit adorned,\nLord Hanuman, the mighty one,\nWith a heart of gold, as bright as the sun.\n\n(Chorus)\nOh Hanuman, embodiment of grace,\nWith courage and love, you light up any space,\nProtector and savior, to you we pray,\nGuide us on this journey, each and every day.\n\n(Verse 2)\nIn the kingdom of Lanka, darkness prevailed,\nRavana's reign of terror, the world was impaled,\nBut Hanuman, with his unwavering might,\nCrossed the ocean, to bring hope and light.\n\n(Chorus)\nOh Hanuman, embodiment of grace,\nWith courage and love, you light up any space,\nProtector and savior, to you we pray,\nGuide us on this journey, each and every day.\n\n(Bridge)\nWith devotion in his heart, he soared through the sky,\nIn search of Sita, held captive, he would not deny,\nWith a ring from Lord Rama, his mission was clear,\nTo deliver a message, to banish all fear.\n\n(Verse 3)\nHanuman, the messen

## Prompt Template + LLM + Output Parsers

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [31]:
class Commaseparatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split('\n')

In [32]:
template = '''
You are a Data Science helpful assistant. So when the user asks you something about data science,machine learning,
deep learning, NLP and generative AI so on this topics you have to guide them properly   
'''

human_template = '{text}'

chat_prompt = ChatPromptTemplate.from_messages([
    ('system',template),
    ('human',human_template)
])

In [39]:
final_chain = chat_prompt|chat_llm|Commaseparatedoutput()

In [40]:
final_chain.invoke({'text':'Roadmap for machine learning'})

["Sure! Here's a roadmap for machine learning:",
 '',
 '1. Learn the basics of programming: Start by learning a programming language like Python or R, which are widely used in machine learning.',
 '',
 "2. Understand the fundamentals of mathematics and statistics: Machine learning heavily relies on concepts such as linear algebra, calculus, probability, and statistics. So, it's important to have a strong foundation in these areas.",
 '',
 '3. Learn about data preprocessing: Familiarize yourself with techniques for cleaning, transforming, and preparing data for analysis. This includes handling missing values, outlier detection, normalization, and feature scaling.',
 '',
 '4. Study supervised learning algorithms: Start with algorithms like linear regression, logistic regression, decision trees, and random forests. Understand how these algorithms work and when to use them.',
 '',
 '5. Dive into unsupervised learning algorithms: Learn about clustering algorithms like K-means, hierarchical 